In [1]:
import cv2
import numpy as np
import open3d as o3d

In [2]:
lidar_data_path = '/Users/austin/Downloads/VRFusion/lidar/chengdu-2025-02-26/data/1740546259.365476000.pcd'
img_data_path = '/Users/austin/Downloads/VRFusion/camera/chengdu-2025-02-26/images/2025-02-26_13:04:19.358_413.png'

In [3]:
def degrees_to_radians(degrees):
    return degrees * np.pi / 180

# 创建绕X轴的旋转矩阵
def rotation_matrix_x(a):
    a = degrees_to_radians(a)
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(a), -np.sin(a)],
        [0, np.sin(a), np.cos(a)]
    ])
    return R_x

# 创建绕Y轴的旋转矩阵
def rotation_matrix_y(b):
    b = degrees_to_radians(b)
    R_y = np.array([
        [np.cos(b), 0, np.sin(b)],
        [0, 1, 0],
        [-np.sin(b), 0, np.cos(b)]
    ])
    return R_y

# 创建绕Z轴的旋转矩阵
def rotation_matrix_z(c):
    c = degrees_to_radians(c)
    R_z = np.array([
        [np.cos(c), -np.sin(c), 0],
        [np.sin(c), np.cos(c), 0],
        [0, 0, 1]
    ])
    return R_z

# R = np.dot(rotation_matrix_z(c), np.dot(rotation_matrix_y(b), rotation_matrix_x(a)))

In [4]:
image = cv2.imread(img_data_path)
pcd = o3d.io.read_point_cloud(lidar_data_path)
points = np.asarray(pcd.points)

# points[:, 0] = -points[:, 0] + 73
# points[:, 1] = -points[:, 1] + 36

points[:, 0] = -points[:, 0]
points[:, 1] = -points[:, 1]

points[:, 2] = 0

In [5]:
camera_matrix = np.array([
    [7552.52547830499, 0.0, 2004.3732920712564],
    [0.0, 7559.4305148647, 1230.2384314670633],
    [0.0, 0.0, 1.0]
])

R = np.array([
    [-0.05678593, 0.99829066, 0.01382437],
    [0.13648184, 0.02147875, -0.9904097],
    [-0.98901368, -0.05435456, -0.13746823]
])

# R = np.array([
#     [-1400.534234, 4190.304943, 1994.684341],
#     [-96.16794156, 103.7901043, 31985.15021],
#     [-0.578113491, -0.006838095, 1.]
# ])

# T = np.array([
#     [-19.23468615],
#     [-3.6991679],
#     [73.8228221]
# ])

# T = np.array([
#     [22.11662476],
#     [-18.00724158],
#     [181.06569838]
# ])

In [6]:
original_image = image.copy()
original_points = points.copy()

In [7]:
def update_transform(val):

    image[:] = original_image

    # angle_x = cv2.getTrackbarPos('x', 'Transformed Image') - 180
    # angle_y = cv2.getTrackbarPos('y', 'Transformed Image') - 180
    # angle_z = cv2.getTrackbarPos('z', 'Transformed Image') - 180

    trans_x = cv2.getTrackbarPos('Vec_T_0', 'Transformed Image') - 1000
    trans_y = cv2.getTrackbarPos('Vec_T_1', 'Transformed Image') - 1000
    trans_z = cv2.getTrackbarPos('Vec_T_2', 'Transformed Image') - 1000

    trans_xx = cv2.getTrackbarPos('PCD_x', 'Transformed Image') - 1000
    trans_yy = cv2.getTrackbarPos('PCD_y', 'Transformed Image') - 1000
    trans_zz = cv2.getTrackbarPos('PCD_z', 'Transformed Image') - 1000

    points[:, 0] = original_points[:, 0] + trans_xx * 0.1
    points[:, 1] = original_points[:, 1] + trans_yy * 0.1
    points[:, 2] = original_points[:, 2] + trans_zz * 0.1

    T = np.array([
        [trans_x * 0.1],
        [trans_y * 0.1],
        [trans_z * 0.1]
    ])

    # R = np.dot(rotation_matrix_z(angle_z), np.dot(rotation_matrix_y(angle_y), rotation_matrix_x(angle_x)))

    point_cloud_camera = R.dot(points.T) + T.reshape(3, 1)
    points_2d = camera_matrix.dot(point_cloud_camera)
    points_2d /= points_2d[2, :]

    points_trans_2d = points_2d[0:2, :].T.astype(np.int32)

    for point in points_trans_2d:
        cv2.circle(image, tuple(point), 5, (0, 0, 255), -1)

    cv2.imshow('Transformed Image', image)

In [ ]:
if image is None:
    print("Error: Image not found.")
else:
    cv2.namedWindow('Transformed Image')

    cv2.createTrackbar('Vec_T_0', 'Transformed Image', 1000, 2000, update_transform)
    cv2.createTrackbar('Vec_T_1', 'Transformed Image', 1000, 2000, update_transform)
    cv2.createTrackbar('Vec_T_2', 'Transformed Image', 1000, 2000, update_transform)
    cv2.createTrackbar('PCD_x', 'Transformed Image', 1000, 2000, update_transform)
    cv2.createTrackbar('PCD_y', 'Transformed Image', 1000, 2000, update_transform)
    cv2.createTrackbar('PCD_z', 'Transformed Image', 1000, 2000, update_transform)

    update_transform(None)

    cv2.waitKey(0)
    cv2.destroyAllWindows()